# Load data in

In [ ]:
import os
from pathlib import Path

current_dir = Path('./')
data_dir = current_dir / 'speech_commands_v0.01'
train_file_path = data_dir / "train_digit_list.txt"
test_file_path = data_dir / "testing_digit_list.txt"

train_file = open(train_file_path, "r")
training_list = [data_dir / x for x in train_file.read().splitlines()]

test_file = open(test_file_path, "r")
testing_list = [data_dir / x for x in test_file.read().splitlines()]

In [ ]:
import os
from pathlib import Path
import numpy as np

with open('X_train_original.npy', 'rb') as f:
    X_train_org = np.load(f)

with open('X_train_reverb_random.npy', 'rb') as f:
    X_train_reverb = np.load(f)
    
with open('X_test_reverb_random.npy', 'rb') as f:
    X_test_reverb = np.load(f)

# GFCC algorithm on 1 data

In [ ]:
import librosa
import matplotlib.pyplot as plt
import numpy as np
from gammatone.plot import gtgram_plot
import gammatone

def emphasize(signal, plot=True):
    pre_emphasis = 0.97
    emphasized_signal = np.append(signal[0], signal[1:] - pre_emphasis * signal[:-1])
    if plot:
        x = np.arange(0, 1., 6.25e-5)
        plt.plot(x, signal, label='original')
        plt.plot(x, emphasized_signal, label='emphasized')
        plt.legend()
        plt.title('Emphasized signal')
        plt.savefig('img/e_signal.png')
        plt.show()


    return emphasized_signal

def extract_gfcc_feature(signal, plot=True):
    e_signal = emphasize(signal, plot)
    samplerate = 16000

    # Default gammatone-based spectrogram parameters
    twin = 0.08
    thop = twin / 2
    channels = 1024
    fmin = 20
    
    gtg = gammatone.gtgram.gtgram(signal, samplerate, twin, thop, channels, fmin)
    Z = np.flipud(20 * np.log10(gtg))

    if plot:
        # Set up the plot
        fig = plt.figure()
        axes = fig.add_axes([0.1, 0.1, 0.8, 0.8])

        gtgram_plot(
            gammatone.gtgram.gtgram,
            axes,
            e_signal,
            samplerate,
            twin, thop, channels, fmin)

        axes.set_xlabel("Time (s)")
        axes.set_ylabel("Frequency")
        plt.savefig('img/gfcc.png')
        plt.show()
    
    
    return Z
    

In [ ]:
data = X_train_reverb[25].flatten()
z = extract_gfcc_feature(data)

In [ ]:
print(z[0])

In [ ]:
data = X_train_org[25].flatten()
extract_gfcc_feature(data)

# Do GFCC on all data

In [ ]:
from tqdm.notebook import tnrange

train_data = []
for i in tnrange(len(X_train_reverb)):
    x = extract_gfcc_feature(X_train_reverb[i].flatten(), plot=False).T
    train_data.append(x)
    
train_data = np.asarray(train_data)

In [ ]:
save_dir = Path('./FE_data')
np.save(save_dir / "GFCC_train_reverb.npy", train_data)

In [ ]:
from tqdm.notebook import tnrange

test_data = []
for i in tnrange(len(X_test_reverb)):
    x = extract_gfcc_feature(X_test_reverb[i].flatten(), plot=False).T
    test_data.append(x)
    
test_data = np.asarray(test_data)

In [ ]:
save_dir = Path('./FE_data')
np.save(save_dir / "GFCC_test_reverb.npy", test_data)